In [7]:
%load_ext autoreload
%autoreload 2

from chess import Game
from chess import Agent
from chess.pieces import *
import random
import time
import cProfile
import copy
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
len(agent._cache)

14125

In [20]:
agent = Agent(depth=4, with_caching=True)
game = Game()

agent.policy(game)

1 cached!
2 cached!
3 cached!
4 cached!
5 cached!
6 cached!
7 cached!
8 cached!
9 cached!
10 cached!
11 cached!
12 cached!
13 cached!
14 cached!
15 cached!
16 cached!
17 cached!
18 cached!
19 cached!
20 cached!
21 cached!
22 cached!
23 cached!
24 cached!
25 cached!
26 cached!
27 cached!
28 cached!
29 cached!
30 cached!
31 cached!
32 cached!
33 cached!
34 cached!
35 cached!
36 cached!
37 cached!
38 cached!
39 cached!
40 cached!
41 cached!
42 cached!
43 cached!
44 cached!
45 cached!
46 cached!
47 cached!
48 cached!
49 cached!
50 cached!
51 cached!
52 cached!
53 cached!
54 cached!
55 cached!
56 cached!
57 cached!
58 cached!
59 cached!
60 cached!
61 cached!
62 cached!
63 cached!
64 cached!
65 cached!
66 cached!
67 cached!
68 cached!
69 cached!
70 cached!
71 cached!
72 cached!
73 cached!
74 cached!
75 cached!
76 cached!
77 cached!
78 cached!
79 cached!
80 cached!
81 cached!
82 cached!
83 cached!
84 cached!
85 cached!
86 cached!
87 cached!
88 cached!
89 cached!
90 cached!
91 cached!
92 cache

723 cached!
724 cached!
725 cached!
726 cached!
727 cached!
728 cached!
729 cached!
730 cached!
731 cached!
732 cached!
733 cached!
734 cached!
735 cached!
736 cached!
737 cached!
738 cached!
739 cached!
740 cached!
741 cached!
742 cached!
743 cached!
744 cached!
745 cached!
746 cached!
747 cached!
748 cached!
749 cached!
750 cached!
751 cached!
752 cached!
753 cached!
754 cached!
755 cached!
756 cached!
757 cached!
758 cached!
759 cached!
760 cached!
761 cached!
762 cached!
763 cached!
764 cached!
765 cached!
766 cached!
767 cached!
768 cached!
769 cached!
770 cached!
771 cached!
772 cached!
773 cached!
774 cached!
775 cached!
776 cached!
777 cached!
778 cached!
779 cached!
780 cached!
781 cached!
782 cached!
783 cached!
784 cached!
785 cached!
786 cached!
787 cached!
788 cached!
789 cached!
790 cached!
791 cached!
792 cached!
793 cached!
794 cached!
795 cached!
796 cached!
797 cached!
798 cached!
799 cached!
800 cached!
801 cached!
802 cached!
803 cached!
804 cached!
805 cached!
806 

1399 cached!
1400 cached!
1401 cached!
1402 cached!
1403 cached!
1404 cached!
1405 cached!
1406 cached!
1407 cached!
1408 cached!
1409 cached!
1410 cached!
1411 cached!
1412 cached!
1413 cached!
1414 cached!
1415 cached!
1416 cached!
1417 cached!
1418 cached!
1419 cached!
1420 cached!
1421 cached!
1422 cached!
1423 cached!
1424 cached!
1425 cached!
1426 cached!
1427 cached!
1428 cached!
1429 cached!
1430 cached!
1431 cached!
1432 cached!
1433 cached!
1434 cached!
1435 cached!
1436 cached!
1437 cached!
1438 cached!
1439 cached!
1440 cached!
1441 cached!
1442 cached!
1443 cached!
1444 cached!
1445 cached!
1446 cached!
1447 cached!
1448 cached!
1449 cached!
1450 cached!
1451 cached!
1452 cached!
1453 cached!
1454 cached!
1455 cached!
1456 cached!
1457 cached!
1458 cached!
1459 cached!
1460 cached!
1461 cached!
1462 cached!
1463 cached!
1464 cached!
1465 cached!
1466 cached!
1467 cached!
1468 cached!
1469 cached!
1470 cached!
1471 cached!
1472 cached!
1473 cached!
1474 cached!
1475 cached!

2045 cached!
2046 cached!
2047 cached!
2048 cached!
2049 cached!
2050 cached!
2051 cached!
2052 cached!
2053 cached!
2054 cached!
2055 cached!
2056 cached!
2057 cached!
2058 cached!
2059 cached!
2060 cached!
2061 cached!
2062 cached!
2063 cached!
2064 cached!
2065 cached!
2066 cached!
2067 cached!
2068 cached!
2069 cached!
2070 cached!
2071 cached!
2072 cached!
2073 cached!
2074 cached!
2075 cached!
2076 cached!
2077 cached!
2078 cached!
2079 cached!
2080 cached!
2081 cached!
2082 cached!
2083 cached!
2084 cached!
2085 cached!
2086 cached!
2087 cached!
2088 cached!
2089 cached!
2090 cached!
2091 cached!
2092 cached!
2093 cached!
2094 cached!
2095 cached!
2096 cached!
2097 cached!
2098 cached!
2099 cached!
2100 cached!
2101 cached!
2102 cached!
2103 cached!
2104 cached!
2105 cached!
2106 cached!
2107 cached!
2108 cached!
2109 cached!
2110 cached!
2111 cached!
2112 cached!
2113 cached!
2114 cached!
2115 cached!
2116 cached!
2117 cached!
2118 cached!
2119 cached!
2120 cached!
2121 cached!

2712 cached!
2713 cached!
2714 cached!
2715 cached!
2716 cached!
2717 cached!
2718 cached!
2719 cached!
2720 cached!
2721 cached!
2722 cached!
2723 cached!
2724 cached!
2725 cached!
2726 cached!
2727 cached!
2728 cached!
2729 cached!
2730 cached!
2731 cached!
2732 cached!
2733 cached!
2734 cached!
2735 cached!
2736 cached!
2737 cached!
2738 cached!
2739 cached!
2740 cached!
2741 cached!
2742 cached!
2743 cached!
2744 cached!
2745 cached!
2746 cached!
2747 cached!
2748 cached!
2749 cached!
2750 cached!
2751 cached!
2752 cached!
2753 cached!
2754 cached!
2755 cached!
2756 cached!
2757 cached!
2758 cached!
2759 cached!
2760 cached!
2761 cached!
2762 cached!
2763 cached!
2764 cached!
2765 cached!
2766 cached!
2767 cached!
2768 cached!
2769 cached!
2770 cached!
2771 cached!
2772 cached!
2773 cached!
2774 cached!
2775 cached!
2776 cached!
2777 cached!
2778 cached!
2779 cached!
2780 cached!
2781 cached!
2782 cached!
2783 cached!
2784 cached!
2785 cached!
2786 cached!
2787 cached!
2788 cached!

3451 cached!
3452 cached!
3453 cached!
3454 cached!
3455 cached!
3456 cached!
3457 cached!
3458 cached!
3459 cached!
3460 cached!
3461 cached!
3462 cached!
3463 cached!
3464 cached!
3465 cached!
3466 cached!
3467 cached!
3468 cached!
3469 cached!
3470 cached!
3471 cached!
3472 cached!
3473 cached!
3474 cached!
3475 cached!
3476 cached!
3477 cached!
3478 cached!
3479 cached!
3480 cached!
3481 cached!
3482 cached!
3483 cached!
3484 cached!
3485 cached!
3486 cached!
3487 cached!
3488 cached!
3489 cached!
3490 cached!
3491 cached!
3492 cached!
3493 cached!
3494 cached!
3495 cached!
3496 cached!
3497 cached!
3498 cached!
3499 cached!
3500 cached!
3501 cached!
3502 cached!
3503 cached!
3504 cached!
3505 cached!
3506 cached!
3507 cached!
3508 cached!
3509 cached!
3510 cached!
3511 cached!
3512 cached!
3513 cached!
3514 cached!
3515 cached!
3516 cached!
3517 cached!
3518 cached!
3519 cached!
3520 cached!
3521 cached!
3522 cached!
3523 cached!
3524 cached!
3525 cached!
3526 cached!
3527 cached!

((1, 7), (3, 7))

In [27]:
agent = Agent(depth=4, with_caching=True)
game = Game()

cProfile.run("agent.policy(game)")

         22405323 function calls (19848840 primitive calls) in 25.471 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   25.471   25.471 <string>:1(<module>)
    12405    0.025    0.000    0.043    0.000 __init__.py:1361(debug)
        1    0.000    0.000    0.000    0.000 __init__.py:1373(info)
    12406    0.018    0.000    0.018    0.000 __init__.py:1619(isEnabledFor)
        1    0.000    0.000    0.000    0.000 __init__.py:183(dumps)
    14125    0.079    0.000    0.399    0.000 agent.py:101(combine)
   126945    0.052    0.000    0.101    0.000 agent.py:105(<lambda>)
        1    0.000    0.000    0.000    0.000 agent.py:27(__str__)
    12385    0.626    0.000    0.652    0.000 agent.py:30(state_value)
        1    0.001    0.001   25.471   25.471 agent.py:41(policy)
 17734/20    0.551    0.000   25.461    1.273 agent.py:61(alphabeta)
    17734    0.034    0.000    3.173    0.000 board.py:14

In [26]:
agent = Agent(depth=4, with_caching=False)
game = Game()

cProfile.run("agent.policy(game)")

         22272627 function calls (21975291 primitive calls) in 24.856 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   24.856   24.856 <string>:1(<module>)
    16661    0.028    0.000    0.049    0.000 __init__.py:1361(debug)
        1    0.000    0.000    0.000    0.000 __init__.py:1373(info)
    16662    0.020    0.000    0.020    0.000 __init__.py:1619(isEnabledFor)
        1    0.000    0.000    0.000    0.000 __init__.py:183(dumps)
    18596    0.093    0.000    0.503    0.000 agent.py:101(combine)
   167184    0.063    0.000    0.122    0.000 agent.py:105(<lambda>)
        1    0.000    0.000    0.000    0.000 agent.py:27(__str__)
    16641    0.765    0.000    0.795    0.000 agent.py:30(state_value)
        1    0.000    0.000   24.856   24.856 agent.py:41(policy)
 18596/20    0.514    0.000   24.844    1.242 agent.py:61(alphabeta)
    18596    0.032    0.000    3.126    0.000 board.py:14